# Global consumer electronic brand sales analysis



## Importing libraries and data

During this step, we import the necessary libraries and load the data required for further analysis. 

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, ForeignKey
from sqlalchemy import Column, Integer, String, inspect, Float, Date, join, select
import os
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
init_notebook_mode(connected=True) 

In [36]:
customers = pd.read_csv('data/Customers.csv', encoding='ISO-8859-1')
exchange_rates = pd.read_csv('data/Exchange_Rates.csv', encoding='ISO-8859-1')
products = pd.read_csv('data/Products.csv', encoding='ISO-8859-1')
sales = pd.read_csv('data/Sales.csv', encoding='ISO-8859-1')
stores = pd.read_csv('data/Stores.csv', encoding='ISO-8859-1')

## Assigning correct datatypes to variables

In this step, we will assign the appropriate data types to the DataFrame columns to ensure optimal performance, memory usage, and accurate analysis.
This version emphasizes the purpose of assigning correct data types and includes the benefits like performance, memory optimization, and data accuracy.

In [37]:
customers.head()

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965


In [38]:
customers['Gender'] = customers['Gender'].astype('category')
customers['Birthday'] = customers['Birthday'].astype('datetime64[ns]')

In [39]:
exchange_rates.head()

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415


In [40]:
exchange_rates['Date'] = exchange_rates['Date'].astype('datetime64[ns]')
exchange_rates['Currency'] = exchange_rates['Currency'].astype('category')

In [41]:
products.head()

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio


In [42]:
products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype('float64')

In [43]:
sales.head()

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD


In [44]:
sales['Order Date'] = sales['Order Date'].astype('datetime64[ns]')
sales['Delivery Date'] = sales['Delivery Date'].astype('datetime64[ns]')


In [45]:
stores.head()

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015


In [46]:
stores['Open Date'] = stores['Open Date'].astype('datetime64[ns]')

## Data Cleaning

This step examines the shape of the datasets and identifies any irregularities or anomalies in the data.

### Exploring the shape of the datasets

In [47]:
customers.shape

(15266, 10)

In [48]:
exchange_rates.shape

(11215, 3)

In [49]:
products.shape

(2517, 10)

In [50]:
sales.shape

(62884, 9)

In [51]:
stores.shape

(67, 5)

The datasets have varying shapes, with the stores dataset being the smallest in terms of observations, while the sales dataset contains the most observations.

### Exploring the number of missing values in each dataset

In [52]:
(customers.isnull().sum(axis=1) > 0).sum()

np.int64(10)

In [53]:
(exchange_rates.isnull().sum(axis=1) > 0).sum()

np.int64(0)

In [54]:
(products.isnull().sum(axis=1) > 0).sum()

np.int64(0)

In [55]:
(sales.isnull().sum(axis=1) > 0).sum()

np.int64(49719)

In [56]:
(stores.isnull().sum(axis=1) > 0).sum()

np.int64(1)

The number of rows with missing values is insignificant in most datasets and can be safely dropped. However, in the sales dataset, out of 63,000 rows, almost 50,000 contain missing values, which makes dropping rows impractical. A better approach would be to identify which variables (columns) in the sales dataset contain the most missing values and decide how to handle them, such as by imputing missing values or analyzing why they are missing.

### Dealing with missing values

In [57]:
customers.dropna(inplace=True)

In [58]:
stores.dropna(inplace=True)

In [59]:
missing_values = sales.isnull().sum()

In [60]:
missing_values

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

The sales dataset contains missing values exclusively in the Delivery Date column, where the dates weren't recorded properly. A potential solution to fill these missing values is to calculate the average delivery time in days and use this value to estimate the missing Delivery Dates by adding the calculated number of days to the corresponding Order Date. This approach will allow for reasonable estimations of the missing Delivery Dates, enabling us to fill in the missing data with a method grounded in the dataset's historical patterns.

In [61]:
average_delivery_time = (sales[-sales['Delivery Date'].isnull()]['Delivery Date'] - sales[-sales['Delivery Date'].isnull()]['Order Date']).mean()

In [62]:
sales.loc[sales['Delivery Date'].isnull(), 'Delivery Date'] = sales.loc[sales['Delivery Date'].isnull(), 'Order Date'] + average_delivery_time

## Setting up SQLite database

Setting up the SQLite database and loading the required data for efficient access.

In [63]:
engine = create_engine("sqlite:///database/my_database.db")
metadata = MetaData()
metadata.bind = engine

In [64]:
# Upload the DataFrames to the SQLite database
customers.to_sql('customers', engine, if_exists='replace', index=False)
exchange_rates.to_sql('exchange_rates', engine, if_exists='replace', index=False)
products.to_sql('products', engine, if_exists='replace', index=False)
sales.to_sql('sales', engine, if_exists='replace', index=False)
stores.to_sql('stores', engine, if_exists='replace', index=False)


66